This is a tutorial notebook to show some utils 

1. Add a csv file with the name "test.csv" in the data/raw folder

In [ ]:
# Use this initial code to work in the notebook as if it were a module, that
# is, to be able to export classes and functions from other subpackages

import os
import sys

package_path = os.path.abspath('.').split(os.sep + 'notebooks')[0]
if package_path not in sys.path:
    sys.path.append(package_path)

In [ ]:
from {{ cookiecutter.project_slug }}.utils import PathManager, read_file, ExecuteQuery, ReadData

# Get absolute paths for files and folders

In [ ]:
file_name = 'test.csv' #File in data/raw/
PathManager().get_abs_path_file(file_name=file_name)

In [ ]:
folder_name = 'processed' #Folder in data/
PathManager().get_abs_path_file(file_name=file_name)

# Read file stored in a folder

In [ ]:
file_name = 'test.csv' #File in data/raw/
read_file(file_path = file_name) # Past the full path or just the file's name

# Read data from Impala using query

In [ ]:
ReadData().from_impala(query='test.sql')

# Execute queries from python

In [ ]:
# Object with connection to Impala. If your ODBS connection has the name 
# 'impala_cloud', work without adding the conn_str parameter, otherwise supply 
# this parameter to the object with the name of your connection

# Note that the object has the 'execute' attribute set to True by default. 
# If you only want to use functionalities without executing the queries, 
# set it to False.

# Note: If you want to do this for each object you create, you can go to the 
# path {}/utils/make_connection.py and update the value of the conn_str 
# parameter on line 8

executer = ExecuteQuery(execute=True, conn_str="DSN=impala_nube")

## Run query in current notebook

In [ ]:
query = """
SELECT *
FROM alianza.binmc
"""

executer.run_sql_query(query=query, verbose=True)

## Run query stored in folder

In [ ]:
# You can directly pass the name of the file containing the query. In this 
# example it is located in the queries/develop folder

file_name = 'test.sql'
executer.run_sql_query(query=file_name, verbose=True)

## Create empty table

In [ ]:
executer. create_empty_table(
                        table_name = 'work_sas.prueba_analitica_riesgos',
                        columns = list('abcde'), # List of columns names
                        types_columns = ["STRING", "STRING", "STRING", "INT", "INT"], # Types of each column. By default all like STRING
                        drop_table = True, # Delete table if exists
                        drop_just_metadata = False, # Execute a TRUNCATE method, use in external tables
                        external_table = True, # Create table as EXTERNAL TABLE 
                        partitioned = True, # Generate partitions in table
                        columns_to_partition = ['a', 'd'],  # Columns to make partitions
                        verbose = True
                    )

## Create table using a query

In [ ]:
executer.create_table_as(
                        table_name = 'work_sas.prueba_analitica_riesgos',
                        as_query = "test.sql", # Past query of .sql file with query
                        drop_table = True, # Delete table if exists
                        drop_just_metadata = False, # Execute a TRUNCATE method, use in external tables
                        external_table = True, # Create table as EXTERNAL TABLE  
                        partitoned = True, # Generate partitions in table
                        columns_to_partition = ['grupo_producto'], # Columns to make partitions
                        verbose = True
                    )

## Insert values using a query

In [ ]:
# Since the INSERT INTO is using a partition on the product_group column, this 
# column has to be removed from the insert_query

# Since you are adding a value to the partition, all records are left with that value

insert_query = read_file('test.sql')
insert_query = ''.join(insert_query.split(',\n    grupo_producto'))

executer.insert_values(table_name = 'work_sas.prueba_analitica_riesgos',
                    type_insert = 'by_query', # You can insert the results of a query or 'by_values'
                    into = True, # Select between INTO or OVERWRITE insert
                    insert_query = insert_query, # The query response values ​​are inserted into the new table
                    values = None, # Just for 'by_values' type insert
                    partitioned = True, # Generate partitions in table
                    columns_to_partition = ['grupo_producto'], 
                    partition_values = ['"MASTERCARD CARULLA"'],
                    verbose = True)

In [ ]:
# If you just want to keep the value of the partition column then keep the 
# parameter partition_values=[], and do not remove the column from the insert_query

insert_query = read_file('test.sql')
# insert_query = ''.join(insert_query.split(',\n    grupo_producto'))

executer.insert_values(table_name = 'work_sas.prueba_analitica_riesgos',
                    type_insert = 'by_query', # You can insert the results of a query or 'by_values'
                    into = True, # Select between INTO or OVERWRITE insert
                    insert_query = insert_query, # The query response values ​​are inserted into the new table
                    values = None, # Just for 'by_values' type insert
                    partitioned = True, # Generate partitions in table
                    columns_to_partition = ['grupo_producto'], 
                    partition_values = [],
                    verbose = True)

## Create table using insert

In [ ]:
# This functionality first creates an empty table with the columns supplied and 
# then inserts values ​​into the table.

# It is a fusion of create_empty_table and insert_values ​​methods.

# Take into account the instructions in the 'Insert values using a query' section 
# since the same conditions apply here regarding insert_values ​​and partition_values


columns_example = [
    'bin',
    'desc_bin',
    'abrev',
    'grupo_bin',
    'logo',
    'grupo_producto'
]

executer.create_table_insert( 
                            table_name = 'work_sas.prueba_analitica_riesgos',
                            columns = columns_example,
                            into = True, # Select between INTO or OVERWRITE insert
                            insert_query = "test.sql", # The query response values ​​are inserted into the new table
                            type_insert = 'by_query', # You can insert the results of a query or 'by_values'
                            values = None, # Just for 'by_values' type insert
                            types_columns = ['STRING'], # By default all columns like STRING
                            drop_table = True, # Delete table if exists
                            drop_just_metadata = False, # Execute a TRUNCATE method, use in external tables
                            external_table = True, # Create table as EXTERNAL TABLE 
                            partitioned = True, # Generate partitions in table 
                            columns_to_partition = ['grupo_producto'], # Columns to generate partitions
                            partition_values = [], # Use it when the results of the insert_query correspond to a specific partition value, for example when you are inserting values ​​into a history and partitioning by 'fa'
                            verbose = True
                        )